# Evaluation of a Pipeline and its Components

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial5_Evaluation.ipynb)

To be able to make a statement about the performance of a question-answering pipeline or any other pipeline haystack, it is important to evaluate it. Furthermore, evaluation allows determining which components of the pipeline can be improved.
The results of the evaluation can be saved as CSV files, which contain all the information to calculate additional metrics later on or inspect individual predictions.

### Prepare environment

#### Colab: Enable the GPU runtime
Make sure you enable the GPU runtime to experience decent speed in this tutorial.
**Runtime -> Change Runtime type -> Hardware accelerator -> GPU**

<img src="https://raw.githubusercontent.com/deepset-ai/haystack/master/docs/_src/img/colab_gpu_runtime.jpg">

In [1]:
# Make sure you have a GPU running
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git

# If you run this notebook on Google Colab, you might need to
# restart the runtime after installing haystack.

     |████████████████████████████████| 2.5 MB 12.2 MB/s 
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-x_63gv_n
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-x_63gv_n
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 14.1 MB 31 kB/s 
     |████████████████████████████████| 2.5 MB 34.9 MB/s 
     |████████████████████████████████| 51 kB 586 kB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 321 kB 33.4 MB/s 
     |████████████████████████████████| 348 kB 52.7 MB/s 
     |████████████████████████████████| 85 kB 4.3 MB/s 
     |████████████████████████████████| 981 kB 55.0 MB/s 
     |████████████████████████████████| 3.0 MB 33.2 MB/s 
     |████████████████████████████████| 78 kB 6.8 MB/s 
     |████████████████████████████████| 5.6 MB 37.2 MB/s 
     |█████████

## Start an Elasticsearch server
You can start Elasticsearch on your local machine instance using Docker. If Docker is not readily available in your environment (eg., in Colab notebooks), then you can manually download and execute Elasticsearch from source.

In [3]:
# If Docker is available: Start Elasticsearch as docker container
# from haystack.utils import launch_es
# launch_es()

# Alternative in Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [1]:
from haystack.modeling.utils import initialize_device_settings

devices, n_gpu = initialize_device_settings(use_cuda=True)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


## Fetch, Store And Preprocess the Evaluation Dataset

In [2]:
from haystack.utils import fetch_archive_from_http

# Download evaluation data, which is a subset of Natural Questions development set containing 50 documents
doc_dir = "../data/nq"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/nq_dev_subset_v2.json.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

INFO - haystack.utils.import_utils -  Found data stored in `../data/nq`. Delete this first if you really want to fetch new data.


False

In [3]:
# make sure these indices do not collide with existing ones, the indices will be wiped clean before data is inserted
doc_index = "tutorial5_docs"
label_index = "tutorial5_labels"

In [4]:
# Connect to Elasticsearch
from haystack.document_stores import ElasticsearchDocumentStore

# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index=doc_index,
                                            label_index=label_index, embedding_field="emb",
                                            embedding_dim=768, excluded_meta_data=["emb"])

In [5]:
from haystack.nodes import PreProcessor

# Add evaluation data to Elasticsearch Document Store
# We first delete the custom tutorial indices to not have duplicate elements
# and also split our documents into shorter passages using the PreProcessor
preprocessor = PreProcessor(
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False
)
document_store.delete_documents(index=doc_index)
document_store.delete_documents(index=label_index)
document_store.add_eval_data(
    filename="../data/nq/nq_dev_subset_v2.json",
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor
)

# Let's prepare the labels that we need for the retriever and the reader
labels = document_store.get_all_labels_aggregated(index=label_index, drop_negative_labels=True, drop_no_answers=False)

## Initialize the Two Components of an ExtractiveQAPipeline: Retriever and Reader

In [6]:
# Initialize Retriever
from haystack.nodes import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)
# Alternative: Evaluate dense Retrievers (DPR and SentenceTransformers)
# Dense Passage Retrieval uses a separate transformer based encoder for query and document each
# SentenceTransformers have a single encoder for both
# Please make sure the "embedding_dim" parameter in the DocumentStore above matches the output dimension of your models!
# Please also take care that the PreProcessor splits your files into chunks that can be completely converted with
#        the max_seq_len limitations of Transformers
# The SentenceTransformer model "all-mpnet-base-v2" generelly works well on any kind of english text.
# For more information check out the documentation at: https://www.sbert.net/docs/pretrained_models.html
# from haystack.retriever import DensePassageRetriever, EmbeddingRetriever
# retriever = DensePassageRetriever(document_store=document_store,
#                                   query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
#                                   passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
#                                   use_gpu=True,
#                                   max_seq_len_passage=256,
#                                   embed_title=True)
# retriever = EmbeddingRetriever(document_store=document_store, model_format="sentence_transformers",
#                                embedding_model="all-mpnet-base-v2")
# document_store.update_embeddings(retriever, index=doc_index)

In [7]:
# Initialize Reader
from haystack.nodes import FARMReader

reader = FARMReader("deepset/roberta-base-squad2", top_k=4, return_no_answer=True)

# Define a pipeline consisting of the initialized retriever and reader
from haystack.pipelines import ExtractiveQAPipeline
pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

# The evaluation also works with any other pipeline.
# For example you could use a DocumentSearchPipeline as an alternative:
# pipeline = DocumentSearchPipeline(retriever=retriever)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.infer -  Got ya 7 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0    0    0    0    0    0    0 
INFO - haystack.modeling.infer -  /w\  /w\  /w\  /w\  /w\  /w\  /w\
INFO - haystack.modeling.i

## Evaluation of an ExtractiveQAPipeline
Here we evaluate retriever and reader in open domain fashion on the full corpus of documents i.e. a document is considered
correctly retrieved if it contains the gold answer string within it. The reader is evaluated based purely on the
predicted answer string, regardless of which document this came from and the position of the extracted span.

The generation of predictions is seperated from the calculation of metrics. So let's start with generating predictions, inspecting them and storing them as an EvaluationResult.


In [8]:
from haystack.schema import Answer, Document, EvaluationResult, Label, MultiLabel, Span

# We can load evaluation labels from the document store
labels_from_doc_store = document_store.get_all_labels()[:5]
eval_labels = [MultiLabel(labels=[label]) for label in labels_from_doc_store]
eval_queries = [label.query for label in labels_from_doc_store]

# Alternative: Define queries and labels directly
# from haystack.schema import Answer, Document, Label, Span
# eval_queries = ["who is written in the book of life"]
# eval_labels = [
#        MultiLabel(labels=[Label(query="who is written in the book of life", answer=Answer(answer="Carla", offsets_in_context=[Span(11, 16)]),
#            document=Document(id='a0747b83aea0b60c4b114b15476dd32d', content_type="text", content='My name is Carla and I live in Berlin'),
#            is_correct_answer=True, is_correct_document=True, origin="gold-label")])
#    ]

# Similar to pipeline.run() we can execute pipeline.eval()
eval_result = pipeline.eval(
    queries=eval_queries,
    labels=eval_labels,
    params={"Retriever": {"top_k": 5}}
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/opt/homebrew/Caskroom/miniforge/base/envs/py39m1_20210816/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /tmp/pip-req-build-q81c20j3/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.31s/ Batches]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.50 Batches/s]


In [9]:
# The EvaluationResult contains a pandas dataframe for each pipeline node.
# That's why there are two dataframes in the EvaluationResult of an ExtractiveQAPipeline.

retriever_result = eval_result["Retriever"]
retriever_result.head()

,content,document_id,type,gold_document_ids,gold_document_contents,gold_id_match,answer_match,gold_id_or_answer_match,rank,node,query
0,"people considered righteous before God. God has such a book, and to be blott...",1b090aec7dbd1af6739c4c80f8995877-1,document,[1b090aec7dbd1af6739c4c80f8995877-0],"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,1,Retriever,who is written in the book of life
1,"as adversaries (of God). Also, according to ib. xxxvi. 10, one who contrives...",1b090aec7dbd1af6739c4c80f8995877-2,document,[1b090aec7dbd1af6739c4c80f8995877-0],"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,2,Retriever,who is written in the book of life
2,the citizens' registers. The life which the righteous participate in is to b...,1b090aec7dbd1af6739c4c80f8995877-6,document,[1b090aec7dbd1af6739c4c80f8995877-0],"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,3,Retriever,who is written in the book of life
3,"apostles' names are ``written in heaven'' (Luke x. 20), or ``the fellow-work...",1b090aec7dbd1af6739c4c80f8995877-3,document,[1b090aec7dbd1af6739c4c80f8995877-0],"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,4,Retriever,who is written in the book of life
4,The Absolutely True Diary of a Part-Time Indian - wikipedia The Absolutely T...,e9260cbbc129f4246ee8fcfbbe385822-0,document,[1b090aec7dbd1af6739c4c80f8995877-0],"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,5,Retriever,who is written in the book of life


In [10]:
reader_result = eval_result["Reader"]
reader_result.head()

,answer,document_id,offsets_in_document,context,type,gold_answers,gold_offsets_in_documents,gold_document_ids,exact_match,f1,rank,node,query
0,,None,"[{'start': 0, 'end': 0}]",None,answer,[every person who is destined for Heaven or the World to Come],"[[Span(start=374, end=434)]]",[1b090aec7dbd1af6739c4c80f8995877-0],0.0,0.000000,1,Reader,who is written in the book of life
1,those whose names are written in the Book of Life from the foundation of the...,1b090aec7dbd1af6739c4c80f8995877-3,"[{'start': 576, 'end': 658}]","ohn of Patmos. As described, only those whose names are written in the Book ...",answer,[every person who is destined for Heaven or the World to Come],"[[Span(start=374, end=434)]]",[1b090aec7dbd1af6739c4c80f8995877-0],0.0,0.083333,2,Reader,who is written in the book of life
2,only the names of the righteous,1b090aec7dbd1af6739c4c80f8995877-1,"[{'start': 498, 'end': 529}]",. The Psalmist likewise speaks of the Book of Life in which only the names o...,answer,[every person who is destined for Heaven or the World to Come],"[[Span(start=374, end=434)]]",[1b090aec7dbd1af6739c4c80f8995877-0],0.0,0.000000,3,Reader,who is written in the book of life
3,those who are found written in the book and who shall escape the troubles pr...,1b090aec7dbd1af6739c4c80f8995877-6,"[{'start': 135, 'end': 305}]",those who are found written in the book and who shall escape the troubles pr...,answer,[every person who is destined for Heaven or the World to Come],"[[Span(start=374, end=434)]]",[1b090aec7dbd1af6739c4c80f8995877-0],0.0,0.111111,4,Reader,who is written in the book of life
0,,None,"[{'start': 0, 'end': 0}]",None,answer,[every person who is destined for Heaven or the World to Come],"[[Span(start=374, end=434)]]",[1b090aec7dbd1af6739c4c80f8995877-0],0.0,0.000000,1,Reader,who is written in the book of life


In [11]:
# We can filter for all documents retrieved for a given query
retriever_berlin = retriever_result[retriever_result['query'] == "Who lives in Berlin?"]

# We can also filter for all answers predicted for a given query
reader_berlin = reader_result[reader_result['query'] == "Who lives in Berlin?"]

# Save the evaluation result so that we can reload it later and calculate evaluation metrics without running the pipeline again.
eval_result.save("../")

## Calculating Evaluation Metrics
Load an EvaluationResult to quickly calculate standard evaluation metrics for all predictions, such as F1-score of each individual prediction of the Reader node or recall of the retriever.

In [14]:
saved_eval_result = EvaluationResult.load("../")
metrics = saved_eval_result.calculate_metrics()
print(metrics["Retriever"]["recall_single_hit"])
print(metrics["Reader"]["f1"])

ValueError: malformed node or string: <ast.Call object at 0x12fd54340>

## Generating an Evaluation Report
A summary of the evaluation results can be printed to get a quick overview. It includes some aggregated metrics and also shows a few wrongly predicted examples.

In [ ]:
pipeline.print_eval_report(saved_eval_result)

## Advanced Evaluation Metrics
As an advanced evaluation metric, semantic answer similarity (SAS) can be calculated. This metric takes into account whether the meaning of a predicted answer is similar to the annotated gold answer rather than just doing string comparison.
To this end SAS relies on pre-trained models. For English, we recommend "cross-encoder/stsb-roberta-large", whereas for German we recommend "deepset/gbert-large-sts". A good multilingual model is "sentence-transformers/paraphrase-multilingual-mpnet-base-v2".
More info on this metric can be found in our [paper](https://arxiv.org/abs/2108.06130) or in our [blog post](https://www.deepset.ai/blog/semantic-answer-similarity-to-evaluate-qa).

In [17]:
EvaluationResult = pipeline.eval(
        queries=EVAL_QUERIES,
        labels=EVAL_LABELS,
        params={"Retriever": {"top_k": 1}},
        sas_model_name_or_path="cross-encoder/stsb-roberta-large"
    )

metrics = eval_result.calculate_metrics()
print(metrics["Reader"]["sas"])

Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.62s/ Batches]


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/139 [00:00<?, ?B/s]

## Evaluation of Individual Components: Retriever
Here we evaluate only the retriever, based on whether the gold_label document is retrieved.

In [ ]:
## Evaluate Retriever on its own
retriever_eval_results = retriever.eval(top_k=20, label_index=label_index, doc_index=doc_index)
# Retriever Recall is the proportion of questions for which the correct document containing the answer is
# among the correct documents
print("Retriever Recall:", retriever_eval_results["recall"])
# Retriever Mean Avg Precision rewards retrievers that give relevant documents a higher rank
print("Retriever Mean Avg Precision:", retriever_eval_results["map"])

## Evaluation of Individual Components: Reader
Here we evaluate only the reader in a closed domain fashion i.e. the reader is given one query
and one document and metrics are calculated on whether the right position in this text is selected by
the model as the answer span (i.e. SQuAD style)

In [ ]:
# Evaluate Reader on its own
reader_eval_results = reader.eval(document_store=document_store, device=devices[0], label_index=label_index, doc_index=doc_index)
# Evaluation of Reader can also be done directly on a SQuAD-formatted file without passing the data to Elasticsearch
#reader_eval_results = reader.eval_on_file("../data/nq", "nq_dev_subset_v2.json", device=device)

# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer
print("Reader Top-N-Accuracy:", reader_eval_results["top_n_accuracy"])
# Reader Exact Match is the proportion of questions where the predicted answer is exactly the same as the correct answer
print("Reader Exact Match:", reader_eval_results["EM"])
# Reader F1-Score is the average overlap between the predicted answers and the correct answers
print("Reader F1-Score:", reader_eval_results["f1"])

## About us

This [Haystack](https://github.com/deepset-ai/haystack/) notebook was made with love by [deepset](https://deepset.ai/) in Berlin, Germany

We bring NLP to the industry via open source!  
Our focus: Industry specific language models & large scale QA systems.  
  
Some of our other work: 
- [German BERT](https://deepset.ai/german-bert)
- [GermanQuAD and GermanDPR](https://deepset.ai/germanquad)
- [FARM](https://github.com/deepset-ai/FARM)

Get in touch:
[Twitter](https://twitter.com/deepset_ai) | [LinkedIn](https://www.linkedin.com/company/deepset-ai/) | [Slack](https://haystack.deepset.ai/community/join) | [GitHub Discussions](https://github.com/deepset-ai/haystack/discussions) | [Website](https://deepset.ai)

By the way: [we're hiring!](https://www.deepset.ai/jobs)